<pre>
Each action has one job
Each Job has one Stage
Each Stage has one or more Tasks running parallely based on Partition 
in the above example as soon as tasks has the Partition tasks increases
</pre>

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession 
from pyspark.conf import SparkConf
import pyspark.sql.functions as F

# Create a SparkConf object
conf = SparkConf().setAppName("learning")\
            .setMaster("local[4]")\
            .set("spark.sql.shuffle.partitions", "2")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

# reading and infering are 2 different actions so its having 2 Jobs 1 stage each stage and 1 task each
df = spark.read\
          .option("header","true")\
          .option("inferSchema","true")\
          .csv("emp_all.txt")



 
partitioned_survey_df = df.repartition(2)   # repartition and write to exchange  one Stage in one partition 1 task  


count_df = partitioned_survey_df.where(F.col("country") > 10)\                   # read exchange  runs in 2 partitions (2 task) wright back to exchange 
                        .select( F.col("name"), F.col("id"), F.col("country"))\  
                        .groupBy(F.col("name"))\
                        .count()

# collect is one Action so having  1 Jobs having  3 stages (1 repartition, 1 count, 1 collect )
print(count_df.collect())            # reads the exhange runs in 2 partition return the results 2 task

input("press Enter")

print("completed")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/24 04:24:48 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp0s3)
25/07/24 04:24:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/24 04:24:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[Row(name='Mitali  ', count=2), Row(name='sohail  ', count=2), Row(name='Manas   ', count=2), Row(name='Saif    ', count=2), Row(name='Ram     ', count=1)]


press Enter hellow


completed


In [2]:

<pre>
the above program has
    3 Jobs    ([reading and infering]   => 2,  [collect] => 1 )
    5 Satages ([reading and infering] ) => 2, [collect] => 3 (1 repartition, 1 count, 1 collect )
    7 Tasks   ([reading and infering] ) => 2, [collect] => 5 (1 repartition, 2 count, 2 collect )

</pre>


+---+--------+-------+
| id|    name|country|
+---+--------+-------+
|101|sohail  | 1000.0|
|102|Saif    | 2000.0|
|103|Mitali  | 3000.0|
|104|Manas   | 4000.0|
|105|Ram     | 5000.0|
|106|sohail  | 1000.0|
|107|Saif    | 2000.0|
|108|Mitali  | 3000.0|
|109|Manas   | 4000.0|
+---+--------+-------+

